# APIs

In [18]:
from elasticsearch import Elasticsearch
import json
from pprint import pprint

In [19]:
es=Elasticsearch(hosts='http://localhost:9200/')
client_info=es.info()
print("Connected to Elastic-Search !")
pprint(client_info.body)

Connected to Elastic-Search !
{'cluster_name': 'docker-cluster',
 'cluster_uuid': 'tGYnSe7ISE2SFo0pu1RcHg',
 'name': '5d9dbd669d35',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2023-02-13T09:35:20.314882762Z',
             'build_flavor': 'default',
             'build_hash': '2d58d0f136141f03239816a4e360a8d17b6d8f29',
             'build_snapshot': False,
             'build_type': 'docker',
             'lucene_version': '9.4.2',
             'minimum_index_compatibility_version': '7.0.0',
             'minimum_wire_compatibility_version': '7.17.0',
             'number': '8.6.2'}}


In [20]:
es.indices.delete(index="test_index", ignore_unavailable=True)

ObjectApiResponse({'acknowledged': True})

In [21]:
res=es.indices.create(index="test_index")

In [22]:
docs=[
    {
        "test_field":"test content1"
    },
    {
        "test_field":"test content2"
    },
    {
        "test_field":"test content3"
    }
]

In [23]:
res_list=[]
for doc in docs:
    res= es.index(index="test_index", document=doc)
    res_list.append(res.body)

In [24]:
res_list

[{'_index': 'test_index',
  '_id': 'ml5vGJUBIxIS9hU_1qHR',
  '_version': 1,
  'result': 'created',
  '_shards': {'total': 2, 'successful': 1, 'failed': 0},
  '_seq_no': 0,
  '_primary_term': 1},
 {'_index': 'test_index',
  '_id': 'm15vGJUBIxIS9hU_16EU',
  '_version': 1,
  'result': 'created',
  '_shards': {'total': 2, 'successful': 1, 'failed': 0},
  '_seq_no': 1,
  '_primary_term': 1},
 {'_index': 'test_index',
  '_id': 'nF5vGJUBIxIS9hU_16Ec',
  '_version': 1,
  'result': 'created',
  '_shards': {'total': 2, 'successful': 1, 'failed': 0},
  '_seq_no': 2,
  '_primary_term': 1}]

## 1. Count

In [27]:
res=es.count(index="test_index")

In [28]:
pprint(res.body)

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'count': 3}


## 2. Exists

In [ ]:
# check if the index exists or not
res=es.indices.exists(index="test_index") # return boolean
pprint(res.body)

True


In [31]:
# check with invalid index
res=es.indices.exists(index="invalid_index") # return boolean
pprint(res.body)

False


In [33]:
# check if the document exists in the index or not

res=es.exists(index=res_list[0]["_index"], id=res_list[0]["_id"])
pprint(res.body)

True
